### Yielding and Generators

먼저 이전 섹션에서 배운 기술을 사용하여 "단순한" Iterator를 작성하는 것으로 시작하겠습니다.

In [1]:
import math

In [2]:
class FactIter:
    def __init__(self, n):
        self.n = n
        self.i = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.i >= self.n:
            raise StopIteration
        else:
            result = math.factorial(self.i)
            self.i += 1
            return result

In [3]:
fact_iter = FactIter(5)

In [4]:
for num in fact_iter:
    print(num)

1
1
2
6
24


We could achieve the same thing using the `iter` method's second form - we just have to know our sentinel value - in this case it would be the factorial of n+1 where n is the last integer's factorial we want our iterator to produce:

In [5]:
def fact():
    i = 0
    def inner():
        nonlocal i
        result = math.factorial(i)
        i += 1
        return result
    return inner           

In [6]:
fact_iter = iter(fact(), math.factorial(5))

In [7]:
for num in fact_iter:
    print(num)

1
1
2
6
24



두 경우 모두 `fact_iter`가 **iterator**였다는 것을 알 수 있습니다. 첫 번째 예에서는 iterator를 직접 구현했고 두 번째 예에서는 python에 bulit-in된 것을 사용했습니다.
두번째 예시는 코드가 조금 적었지만, 만약 우리가 직접 작성하지 않고 코드를 보여준다면 이해하기가 조금 더 어려울 수도 있습니다.
더 좋은 방법이 있을까요?

And indeed, there is... generators.

먼저 `yield`를 봅시다.

`yield`문은 함수의 `return`문 처럼 사용되지만 큰 차이가 있습니다. `yield`문을 만나면 파이썬은 `yield`가 지정한 값을 모두 반환하지만 함수 실행을 **일시정지**합니다. 그런 다음 동일한 함수를 다시 **호출**하면 마지막 `yield`가 발생한 위치에서 다시 재개할 수 있습니다.
저는 call 이라고 부릅니다 왜냐면 함수를 불러도 재개하지 않기 때문입니다.
대신에 우리는 `next` 함수를 사용합니다.

시도해봅시다.

In [8]:
def my_func():
    print('line 1')
    yield 'Flying'
    print('line 2')
    yield 'Circus'    

In [9]:
my_func()

<generator object my_func at 0x0000019DA77D3BA0>

따라서 `my_func()`를 실행하면 제너레이터 개체가 반환됩니다. - 실제로 `my_func`의 본문을 "실행"하지는 않았습니다 (print 중 실제로 실행된 것은 없습니다).

그러기 위해서는 `next()` 함수를 사용해야 합니다.

`next()`?? 그건 반복해야할때 사용하는거 아니야?

In [10]:
gen_my_func = my_func()

In [11]:
next(gen_my_func)

line 1


'Flying'

In [12]:
next(gen_my_func)

line 2


'Circus'

And let's call it one more time:

In [13]:
next(gen_my_func)

StopIteration: 

`StopIteration` exception.

흠... `next`, `StopIteration`? 어떻게 생겼나요?

**iterator**!

그리고 사실 그것이 바로 Python generator입니다. ***그들은*** generator입니다.

generators가 iterators인 경우 iterator **protocol**를 구현해야 합니다.

봅시다.

In [14]:
gen_my_func = my_func()

In [15]:
'__iter__' in dir(gen_my_func)

True

In [16]:
'__next__' in dir(gen_my_func)

True

그래서 우리는 다른 iterator들처럼 `iter()` 함수와 `next()` 함수와 함께 사용할 수 있는 iterator만 있다:

In [17]:
gen_my_func

<generator object my_func at 0x0000019DA78660A0>

In [18]:
iter(gen_my_func)

<generator object my_func at 0x0000019DA78660A0>

보시다시피 `iter` 함수는 같은 객체를 반환했는데 이는 우리가 iterator에서 예상하는 것이다.

So if this is an iterator that Python builds, how does it know when to stop the iteration (raise the `StopIteration` exception)?

In the example above, it seemed clear - when the function finished running - there were no more statements after that last `yield`.

What actually happens if a function finishes running and we don't explicitly return something?

Remember that Python fills in the gap, and returns `None`.

In general, the iteration will terminate when we **return** something from the function.

Let's take a look:

In [19]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            result = i**2
            i += 1
            yield result
        else:
            return 'all done!'

In [20]:
sq = squares(3)

In [21]:
next(sq)

0

In [22]:
next(sq)

1

In [23]:
next(sq)

4

In [24]:
next(sq)

StopIteration: all done!

And the return value of our function became the message of the `StopIteration` exception.

그러나 다음과 같이 약간 단순화 할 수 있습니다.

In [25]:
def squares(sentinel):
    i = 0
    while True:
        if i < sentinel:
            yield i**2
            i += 1 # note how we can incremenet **after** the yield
        else:
            return 'all done!'

In [26]:
for num in squares(5):
    print(num)

0
1
4
9
16


So now let's see how we could re-write our initial `factorial` example:

In [27]:
def factorials(n):
    for i in range(n):
        yield math.factorial(i)    

In [28]:
for num in factorials(5):
    print(num)

1
1
2
6
24


Now that's a much simpler and understandable way to create the iterator!

Note that a generator **is** an iterator, but not vice-versa - iterators are not necessarily generators, just like sequences are iterables, but iterables are not necessarily sequences.

Another thing to note is that since generators are iterators, they also  become exhausted (consumed) just like an iterator does.

In [29]:
facts = factorials(5)

In [30]:
list(facts)

[1, 1, 2, 6, 24]

In [31]:
list(facts)

[]

As you can see, our second iteration through the same generator ended up with nothing - that's because the generator has been exhausted:

In [32]:
next(facts)

StopIteration: 